In [27]:
from stix2 import Bundle,parse
from stix2.exceptions import ExtraPropertiesError,InvalidValueError,ParseError
import json
import glob

In [46]:

for filename in glob.glob('./playbooks/*.json'):
    print('Loading playbook ' + filename)
    with open(filename,'r') as file:
        bundle = json.load(file)
        to_delete = {}
        new_objs = []
        for obj in bundle['objects']:
            try:
                obj_ok = parse(obj,allow_custom=False)
                new_objs.append(obj_ok)
            except ExtraPropertiesError as e:
                #remove the custom fields
                obj_fix = {key: obj_ok[key] for key in obj_ok if key not in e.properties}
                new_objs.append(obj_ok)
            except InvalidValueError as e:
                #remove the entire object because is broken
                to_delete[obj['id']]=obj['type']
            except ParseError as e:
                to_delete[obj['id']]=obj['type']
                
        if len(to_delete) > 0:
            # time to clean up some stuff
            for obj in bundle['objects']:
                if obj['type']=='relationship':
                    if obj['source_ref'] in to_delete.keys():
                        to_delete[obj['id']]=obj['type']
                    if obj['target_ref'] in to_delete.keys():
                        to_delete[obj['id']]=obj['type']
                if obj['type']=='sighting':
                    if obj['sighting_of_ref'] in to_delete.keys():
                        to_delete[obj['id']]=obj['type']
                        
                    if 'observed_data_refs' in obj:
                        for ref in obj['observed_data_refs']:
                            if ref in to_delete.keys():
                                to_delete[obj['id']]=obj['type']
                        
        norm_filename = filename.replace('playbooks','standard')
        print('Rewriting playbook ' + filename)
        with open(norm_filename,'w') as newfile:
            write_objs = []
            
            for obj in new_objs:
                if obj['id'] not in to_delete:
                    write_objs.append(obj)
                    
            new_bundle = Bundle(write_objs,allow_custom=False)

            newfile.write(new_bundle.serialize())
        print('...Done')

Loading playbook ./playbooks/emotet.json
Rewriting playbook ./playbooks/emotet.json
...Done
Loading playbook ./playbooks/tridentursa.json
Rewriting playbook ./playbooks/tridentursa.json
...Done
Loading playbook ./playbooks/ransom-cartel.json
Rewriting playbook ./playbooks/ransom-cartel.json
...Done
Loading playbook ./playbooks/very-simple-playbook.json
Rewriting playbook ./playbooks/very-simple-playbook.json
...Done
Loading playbook ./playbooks/owassrf-vulnerability-exploitation.json
Rewriting playbook ./playbooks/owassrf-vulnerability-exploitation.json
...Done
Loading playbook ./playbooks/evasive-serpens.json
Rewriting playbook ./playbooks/evasive-serpens.json
...Done
Loading playbook ./playbooks/conti-ransomware.json
Rewriting playbook ./playbooks/conti-ransomware.json
...Done
Loading playbook ./playbooks/radioserpens.json
Rewriting playbook ./playbooks/radioserpens.json
...Done
Loading playbook ./playbooks/mespinoza-ransomware.json
Rewriting playbook ./playbooks/mespinoza-ransomware